In [48]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/train.csv', index_col = 0, dtype={'multiple_full_time_jobs': 'boolean', 'combined_multiple_jobs': 'boolean'}).reset_index(drop=True)
strCols = data.select_dtypes(object).columns
#data = data.replace(r'^\s*$', np.nan, regex=True)
for i in strCols:
    data[i] = data[i].str.strip()
#data = data.sample(frac=0.1)
#data = data.fillna({'MI': 'NA'}).fillna(True)
data.HIREDT = data.HIREDT.apply(pd.to_datetime, format = "%m/%d/%y")
data['daysexperience'] = (pd.to_datetime('10/7/23', format = "%m/%d/%y") - data.HIREDT ).dt.days
data_num = ['RATE', 'HRSWKD', 'daysexperience']
data_drop = ['AGY', 'HIREDT', 'JOBCLASS']#, 'multiple_full_time_jobs', 'combined_multiple_jobs']
data.drop(data_drop, axis=1, inplace=True)

important_features = ['daysexperience', 'RATE', 'RACE', 'SEX', 'MI', 'HRSWKD', 'JC.TITLE']


In [58]:
data.SEX = data.SEX.eq('MALE').mul(1)

In [52]:
pd.set_option("display.max_rows", None)
data.NAME.value_counts()

HEALTH AND HUMAN SERVICES COMMISSION                  26622
TEXAS DEPARTMENT OF CRIMINAL JUSTICE                  24726
TEXAS DEPARTMENT OF TRANSPORTATION                    10272
DEPARTMENT OF FAMILY AND PROTECTIVE SERVICES           9594
DEPARTMENT OF PUBLIC SAFETY                            8211
TEXAS WORKFORCE COMMISSION                             3477
OFFICE OF THE ATTORNEY GENERAL                         3066
DEPARTMENT OF STATE HEALTH SERVICES                    2792
PARKS AND WILDLIFE DEPARTMENT                          2412
TEXAS COMMISSION ON ENVIRONMENTAL QUALITY              2124
COMPTROLLER OF PUBLIC ACCOUNTS                         2031
TEXAS JUVENILE JUSTICE DEPARTMENT                      1492
TEXAS DEPARTMENT OF INSURANCE                           933
COMPTROLLER OF PUBLIC ACCOUNTS, JUDICIARY SECTION       930
TEXAS EDUCATION AGENCY                                  857
TEACHER RETIREMENT SYSTEM                               856
HOUSE OF REPRESENTATIVES                

In [59]:
import rpy2
from rpy2.robjects.packages import importr
import rpy2.robjects as robjects
import rpy2.robjects.pandas2ri as pandas2ri

with robjects.conversion.localconverter(
    robjects.default_converter + pandas2ri.converter):
    rdf = robjects.conversion.py2rpy(data)

In [40]:
rdf

NAME,MI,JC.TITLE,...,multiple_full_time_jobs,combined_multiple_jobs,daysexperience
'TEXAS HI...,'C','PROJECT ...,...,1,1,310
'SENATE','NA','LEGISLAT...,,1,1,6820
'COMPTROL...,'J',"'JUDGE, R...",,1,1,1344
'SCHOOL F...,'T','LIBRARY ...,,1,1,8766
...,...,...,,...,...,...
"""STATE AU...",'T','AUDITOR ...,,1,1,279
"""STATE AU...",'H','AUDITOR ...,,1,1,279
"""STATE AU...",'N','AUDITOR ...,,1,1,279
'TEXAS CO...,'M','TRAINING...,,1,1,250


In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112593 entries, 0 to 112592
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   NAME                     112593 non-null  object 
 1   MI                       112593 non-null  object 
 2   JC.TITLE                 112593 non-null  object 
 3   RACE                     112593 non-null  object 
 4   SEX                      112593 non-null  object 
 5   EMPTYPE                  112593 non-null  object 
 6   RATE                     112593 non-null  float64
 7   HRSWKD                   112593 non-null  float64
 8   ANNUAL                   112593 non-null  float64
 9   STATENUM                 112593 non-null  int64  
 10  multiple_full_time_jobs  112593 non-null  boolean
 11  combined_multiple_jobs   112593 non-null  boolean
 12  daysexperience           112593 non-null  int64  
dtypes: boolean(2), float64(3), int64(2), object(6)
memory usage

In [60]:
r_package_matchit = robjects.packages.importr('MatchIt')
func_matchit = robjects.r['matchit']
from rpy2.robjects import Formula

func_matchit(formula=Formula('SEX ~ RATE + HRSWKD + STATENUM'),
             data=rdf, method=robjects.rinterface.NULL,
             distance='glm')

<rpy2.robjects.vectors.ListVector object at 0x286435990> [19]
R classes: ('matchit',)
[FloatSexp..., ListSexpV..., LangSexpV..., ListSexpV..., ..., IntSexpVe..., FloatSexp..., BoolSexpV..., ListSexpV...]
  weights: <class 'rpy2.robjects.vectors.FloatVector'>
  <rpy2.robjects.vectors.FloatVector object at 0x2858da310> [14]
R classes: ('numeric',)
[1.000000, 1.000000, 1.000000, 1.000000, ..., 1.000000, 1.000000, 1.000000, 1.000000]
<rpy2.robjects.vectors.ListVector object at 0x286435990> [19]
R classes: ('matchit',)
[FloatSexp..., ListSexpV..., LangSexpV..., ListSexpV..., ..., IntSexpVe..., FloatSexp..., BoolSexpV..., ListSexpV...]
  call: <class 'rpy2.robjects.language.LangVector'>
  Rlang( (function (formula, data = NULL, method = "nearest", distance = "glm",  )
<rpy2.robjects.vectors.ListVector object at 0x286435990> [19]
R classes: ('matchit',)
[FloatSexp..., ListSexpV..., LangSexpV..., ListSexpV..., ..., IntSexpVe..., FloatSexp..., BoolSexpV..., ListSexpV...]
...
  formula: <class 'rpy2.robjects.vectors.IntVector'>
  <rpy2.robjects.vectors.IntVector object at 0x2871c6d90> [13]
R classes: ('integer',)
[0, 0, 1, 1, ..., 1, 1, 0, 0]
  treat: <class 'rpy2.robjects.vectors.FloatVector'>
  <rpy2.robjects.vectors.FloatVector object at 0x2871c7cd0> [14]
R classes: ('numeric',)
[0.426242, 0.428741, 0.702385, 0.376655, ..., 0.420200, 0.420199, 0.420196, 0.420189]
  distance: <class 'rpy2.robjects.vectors.BoolVector'>
  <rpy2.robjects.vectors.BoolVector object at 0x2871c6510> [10]
R classes: ('logical',)
[       0,        0,        0,        0, ...,        0,        0,        0,        0]
<rpy2.robjects.vectors.ListVector object at 0x286435990> [19]
R classes: ('matchit',)
[FloatSexp..., ListSexpV..., LangSexpV..., ListSexpV..., ..., IntSexpVe..., FloatSexp..., BoolSexpV..., ListSexpV...]